In [85]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from urllib.request import urlopen
import requests

# Scraping Data from a Dynamic Webpage

## Initializing Driver and Loading Webpage

In [86]:
# Specify Options for Driver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)")
options.add_argument("--window-size=1920,1080")

# Instantiate Selenium Driver Object
driver = webdriver.Chrome(options=options)

# Load Webpage
url = 'https://www.inmotionhosting.com/'
driver.get(url)

## Scraping 

In [87]:
# Set Time to Ensure the Data Has Time to Load
wait = WebDriverWait(driver, 10) # 10 seconds

# Wait until the element with ID 'home-rostrum-2' is present and visible
parent = wait.until(EC.presence_of_element_located((By.ID, "home-rostrum-2")))

# Once it’s confirmed to exist, find the nested elements
details = parent.find_elements(By.CLASS_NAME, "imh-rostrum-card")

# Extract necessary data such as plan names, features, and pricing
data = []

# Loop through each plan in the extracted list of plans
for d in details:

    # Extract Name from Plan
    name = d.find_element(By.TAG_NAME, "h3").text

    # Extract List of Features from Plan (Make sure its a comma separated string for CSV format)
    features = ", ".join([fl.text for fl in d.find_elements(By.TAG_NAME, "li")])

    # Extract Price from Plan
    price = float(d.find_element(By.CLASS_NAME, "active.imh-switcher").find_element(By.CLASS_NAME, "rostrum-price").text[1:])

    # Append Each Extracted Piece of Information as a single row (row = dictionary) in the data list
    data.append({
        "plan_name" : name,
        "plan_features" : features,
        "plan_price" : price
    })

# Check
data

[{'plan_name': 'Shared Hosting',
  'plan_features': 'Free Domain & SSL, Free Website Builder, Unmetered Bandwidth, cPanel Included, Free Email',
  'plan_price': 2.99},
 {'plan_name': 'Hosting for WordPress',
  'plan_features': 'Free Domain & SSL, Free Website Builder, Free Premium Themes & Plugins, PHP-FPM Workers Included, Free Email',
  'plan_price': 3.49},
 {'plan_name': 'VPS Hosting',
  'plan_features': 'High-Availability Servers, cPanel and Control Web Panel Available, Free Website Transfers (with a Control Panel), Cloud-Powered Reliability, Isolated Resources with Hardened Security',
  'plan_price': 14.99},
 {'plan_name': 'Dedicated Hosting',
  'plan_features': 'Configurable Server For Ultimate Flexibility, Redundant Infrastructure With 99.99% Uptime, Choose Between Managed With cPanel or Unmanaged Bare Metal',
  'plan_price': 35.0}]

In [88]:
# Close Driver
driver.quit()

# Store And Save Data

In [89]:
# Convert to dataframe and export to csv 
pd.DataFrame(data).to_csv("hosting_plans.csv", index=False)